In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from awpy import Demo
import os
from demoparser2 import DemoParser

In [3]:
parser = DemoParser("..\\..\\demos\\pov\\1.dem")
max_tick = parser.parse_event("round_end")["tick"].max()

wanted_fields = ["kills_total", "deaths_total", "mvps", "headshot_kills_total", "ace_rounds_total", "4k_rounds_total", "3k_rounds_total"]
df = parser.parse_ticks(wanted_fields, ticks=[max_tick]).drop_duplicates()
df

Exception: EntityNotFound

In [4]:
dem = Demo(file="..\\..\\demos\\pov\\1.dem")

Exception: EntityNotFound

In [45]:
def get_coaches(dem):
    df = dem.rounds.merge(dem.ticks, left_on="start", right_on="tick")
    coaches = df.groupby("name").agg({"health":"sum"}).query("health == 0")

    return list(coaches.index)

coaches = get_coaches(dem)

In [61]:
dem.ticks = dem.ticks.query("name not in @coaches")
dem.ticks = dem.ticks.drop_duplicates(subset=['tick', 'name'], keep='last')

In [80]:
dem.ticks["players"] = dem.ticks.groupby("tick")["name"].transform("count")
dem.ticks.query("players > 10")

,inventory,is_bomb_planted,ping,rank,armor,health,weapon,has_defuser,has_helmet,in_crouch,...,pitch,yaw,X,Y,Z,tick,steamid,name,round,players


In [75]:
dem.weapon_fires.player_name.unique()

array([None, 'stavn', 'zFgiveN', 'Staehr', '123'], dtype=object)

In [70]:
kills = dem.kills[["tick","round","attacker_name","victim_name","assister_name"]]

In [91]:
dem.weapon_fires.query("tick == 13636")

,tick,player_name,player_steamid,player_team_name,player_team_clan_name,player_X,player_Y,player_Z,player_yaw,player_pitch,player_last_place_name,player_is_strafing,player_accuracy_penalty,player_health,player_armor,player_rank,player_zoom_lvl,player_inventory,weapon,round
261,13636,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,None,weapon_usp_silencer,1


In [87]:
dem.ticks.query("tick == 13636")

,inventory,is_bomb_planted,ping,rank,armor,health,weapon,has_defuser,has_helmet,in_crouch,...,pitch,yaw,X,Y,Z,tick,steamid,name,round,players
369778,[],False,37,0,0.0,0,16777215,False,False,None,...,1.946991,78.791870,-2754.320312,-1703.700806,473.462982,13636,76561198060483793,br0,1,10
369779,[],False,37,0,0.0,0,16777215,False,False,None,...,4.461487,57.411469,-3113.166992,-1826.526367,518.634216,13636,76561198022365638,stavn,1,10
369780,[],False,57,0,NaN,0,16777215,False,False,False,...,-64.129944,-14.771805,-3547.466553,-2275.996826,479.879761,13636,76561198129084615,clax 1XBET,1,10
369782,[],False,43,0,NaN,0,16777215,False,False,False,...,-14.916000,-124.239922,-1856.833862,-1028.677734,148.489395,13636,76561197963914156,seized,1,10
369784,"[knife_butterfly, Smoke Grenade, C4]",False,37,0,100.0,100,9897131,False,False,None,...,23.971481,49.988831,-1383.940796,-704.521301,9.341045,13636,76561198005107817,Staehr,1,10
369785,[],False,45,0,NaN,0,16777215,False,False,False,...,0.299042,-23.796387,-3589.192627,-2057.500732,482.007935,13636,76561198859428279,d1Ledez,1,10
369786,"[knife_m9_bayonet, Smoke Grenade, Flashbang, M...",False,37,0,0.0,100,10126417,False,False,None,...,2.221985,94.705933,-2591.183350,-1854.585815,476.121857,13636,76561197987713664,zFgiveN,1,10
369787,"[knife_m9_bayonet, USP-S]",False,37,0,100.0,100,2834967,False,False,None,...,8.672333,55.543442,-1369.999512,-825.729797,22.090450,13636,76561198120557348,123,1,10
369788,"[knife_butterfly, USP-S]",False,36,0,NaN,8,377106,False,False,None,...,-1.158371,-97.756004,-2637.556641,-912.323364,436.031250,13636,76561198170649897,iDISBALANCE,1,10
369790,[],False,31,0,NaN,0,16777215,False,False,False,...,3.832520,-51.769989,-2021.546509,-867.827576,132.031250,13636,76561198174263909,glowiing,1,10


In [81]:
kills.head(10)

,tick,round,attacker_name,victim_name,assister_name
41,12808,1,stavn,None,None
42,12885,1,Staehr,None,None
43,12944,1,stavn,None,None
44,13337,1,Staehr,None,None
45,13511,1,None,stavn,None
46,13636,1,None,None,None
47,14177,1,zFgiveN,None,None
48,17504,2,Staehr,None,stavn
49,17533,2,Staehr,None,stavn
50,17537,2,None,Staehr,None
